In [ ]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

user_sql = 'victorodg'
pass_sql = '123456'
server = 'SQNPRC001'
database = 'DBDV910'

string_connect = 'mssql+pyodbc://' + user_sql + ':' + pass_sql + '@' + server + '/' + database + '?driver=SQL+Server'
engine = create_engine(string_connect,echo=False,pool_size=50,max_overflow=50)